# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import *
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
EPOCHS = 4000
initial_number_of_circuits = 196
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:initial_number_of_circuits + 1]
current_training_circuits = {}
result_file = workload + "_" + workload_size + "_noisyopt_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
if initial_number_of_circuits > 5 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

Loading parameters from file execution_time_main_noisyopt_2_1_3


In [ ]:
for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    train_pred_fn = jit(make_pred_fn(training_circuits_l))
    dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
    test_pred_fn = make_pred_fn(test_circuits_l)
    
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
    
    def callback_fn(xk):
        #print(xk)
        valid_loss = dev_cost_fn(xk)
        train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
        train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
        valid_acc = numpy.around(float(dev_accs[-1]), 4)
        iters = int(len(train_accs)/2)
        if iters % 200 == 0:
            info = f"Epoch: {iters}   "\
            + f"train/loss: {train_loss}   "\
            + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
            + f"train/acc: {train_acc}   "\
            + f"valid/acc: {valid_acc}"
        
            with open("results//" + result_file + ".txt", "a") as f:
                f.write(info + "\n")
                
            print(info, file=sys.stderr)
        return valid_loss
    
    a_value = 0.0053
    c_value = 0.0185
            
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

    result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
    #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

    figure_path = this_folder + "//results//" + result_file + ".png"
    visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    #EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.438
Number of training circuits: 196   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 66.8903   valid/loss: 38.7929   train/acc: 0.2959   valid/acc: 0.3097
Epoch: 400   train/loss: 63.1001   valid/loss: 39.0742   train/acc: 0.3469   valid/acc: 0.354
Epoch: 600   train/loss: 61.93   valid/loss: 39.463   train/acc: 0.3163   valid/acc: 0.2478
Epoch: 800   train/loss: 61.5026   valid/loss: 40.4259   train/acc: 0.3827   valid/acc: 0.3097
Epoch: 1000   train/loss: 57.9727   valid/loss: 40.6657   train/acc: 0.4133   valid/acc: 0.3097
Epoch: 1200   train/loss: 56.2942   valid/loss: 39.8684   train/acc: 0.4184   valid/acc: 0.2832
Epoch: 1400   train/loss: 54.922   valid/loss: 41.8839   train/acc: 0.5   valid/acc: 0.3363
Epoch: 1600   train/loss: 54.0102   valid/loss: 41.5122   train/acc: 0.4439   valid/acc: 0.354
Epoch: 1800   train/loss: 54.1731   valid/loss: 41.7199   train/acc: 0.4796   valid/acc: 0.354
Epoch: 2000   train/loss: 53.6623   valid/loss: 43.9395   train/acc: 0.4592   valid/acc: 0.3451
Epoch: 2200   train/loss: 53.5466   valid/loss: 43.942

Test accuracy: 0.42342342342342343
Progress:  0.44
Number of training circuits: 197   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 63.9076   valid/loss: 38.4543   train/acc: 0.3604   valid/acc: 0.3097
Epoch: 400   train/loss: 66.7435   valid/loss: 41.0337   train/acc: 0.3249   valid/acc: 0.2566
Epoch: 600   train/loss: 65.3227   valid/loss: 40.7874   train/acc: 0.2944   valid/acc: 0.2212
Epoch: 800   train/loss: 59.0052   valid/loss: 41.0238   train/acc: 0.401   valid/acc: 0.3451
Epoch: 1000   train/loss: 56.2989   valid/loss: 40.8654   train/acc: 0.4873   valid/acc: 0.3628
Epoch: 1200   train/loss: 54.7871   valid/loss: 41.5348   train/acc: 0.4721   valid/acc: 0.354
Epoch: 1400   train/loss: 52.6661   valid/loss: 41.6746   train/acc: 0.4924   valid/acc: 0.3363
Epoch: 1600   train/loss: 50.4248   valid/loss: 40.087   train/acc: 0.5381   valid/acc: 0.3451
Epoch: 1800   train/loss: 50.2782   valid/loss: 40.0424   train/acc: 0.5431   valid/acc: 0.3363
Epoch: 2000   train/loss: 49.9365   valid/loss: 40.2114   train/acc: 0.5533   valid/acc: 0.3451
Epoch: 2200   train/loss: 48.2623   valid/loss:

Test accuracy: 0.5045045045045045
Progress:  0.442
Number of training circuits: 198   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


...

Test accuracy: 0.4864864864864865
Progress:  0.46
Number of training circuits: 206   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 67.0965   valid/loss: 39.4646   train/acc: 0.3883   valid/acc: 0.3363
Epoch: 400   train/loss: 64.0765   valid/loss: 41.2453   train/acc: 0.4029   valid/acc: 0.2743
Epoch: 600   train/loss: 60.1462   valid/loss: 40.3922   train/acc: 0.4369   valid/acc: 0.2566
Epoch: 800   train/loss: 62.7677   valid/loss: 43.4943   train/acc: 0.4806   valid/acc: 0.2832
Epoch: 1000   train/loss: 59.1941   valid/loss: 39.634   train/acc: 0.4369   valid/acc: 0.3451
Epoch: 1200   train/loss: 57.7327   valid/loss: 42.2481   train/acc: 0.466   valid/acc: 0.2478
Epoch: 1400   train/loss: 56.1065   valid/loss: 39.9468   train/acc: 0.4563   valid/acc: 0.2655
Epoch: 1600   train/loss: 55.7206   valid/loss: 39.7745   train/acc: 0.4903   valid/acc: 0.2832
Epoch: 1800   train/loss: 54.7597   valid/loss: 40.5172   train/acc: 0.4854   valid/acc: 0.3097
Epoch: 2000   train/loss: 53.3158   valid/loss: 39.6851   train/acc: 0.5485   valid/acc: 0.3097
Epoch: 2200   train/loss: 51.5786   valid/loss

Test accuracy: 0.40540540540540543
Progress:  0.462
Number of training circuits: 207   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 68.2675   valid/loss: 40.9766   train/acc: 0.3478   valid/acc: 0.2655
Epoch: 400   train/loss: 66.9812   valid/loss: 39.8883   train/acc: 0.3768   valid/acc: 0.2832
Epoch: 600   train/loss: 63.9867   valid/loss: 41.2892   train/acc: 0.4155   valid/acc: 0.3009
Epoch: 800   train/loss: 60.8923   valid/loss: 45.9975   train/acc: 0.3478   valid/acc: 0.2566
Epoch: 1000   train/loss: 58.7514   valid/loss: 40.6245   train/acc: 0.4589   valid/acc: 0.354
Epoch: 1200   train/loss: 59.0731   valid/loss: 40.2011   train/acc: 0.4976   valid/acc: 0.354
Epoch: 1400   train/loss: 58.9126   valid/loss: 41.1928   train/acc: 0.4879   valid/acc: 0.3805
Epoch: 1600   train/loss: 58.8762   valid/loss: 40.6028   train/acc: 0.4686   valid/acc: 0.3274
Epoch: 1800   train/loss: 59.4222   valid/loss: 40.4585   train/acc: 0.401   valid/acc: 0.3097
Epoch: 2000   train/loss: 56.1754   valid/loss: 40.2181   train/acc: 0.4928   valid/acc: 0.3274
Epoch: 2200   train/loss: 55.5723   valid/loss:

Test accuracy: 0.5405405405405406
Progress:  0.464
Number of training circuits: 208   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
